In [2]:
import pandas as pd
import numpy as np
import operator
import json
import datetime
from datetime import datetime
import itertools
from tqdm import tqdm

In [3]:
# df = pd.read_csv('/users/s0p00zp/Downloads/analysis_38507735-A182-4F98-90C4-08D762F92557_12012019-12062019.csv',
#                header=None)

In [4]:
#folder_path= '/Users/s0c02nj/Downloads/Analysis 3000001 - 4000000.txt'

data = open(folder_path,"r+",encoding="utf8").readlines()
df = pd.DataFrame([sub.split("\t") for sub in data])
df[3] = df[3].str.rstrip('\n')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/s0c02nj/Downloads/Analysis 3000001 - 4000000.txt'

In [4]:
# s = "2016-03-26T09:25:55.000Z"
# f = "%Y-%m-%dT%H:%M:%S.%fZ"

In [5]:
#int(float(df[3][0] .split(":")[1]))

In [6]:
# h=[int(x.split(":")[0]) for x in df[3]]
# minute=[int(float(x.split(":")[1])) for x in df[3]]
# second=[float(x.split(":")[1])-int(float(x.split(":")[1])) for x in df[3]]

In [7]:
# time_stamp=[]
# for (a,b,c) in zip(h,minute,second):
#     print(a,b,c)
#     tmp=datetime.time(a,b,int(c*60)).strftime('%H:%M:%S')
#     time_stamp.append(tmp)
x = df[3].iloc[0]
y = x.split('.')[0]
y
datetime_object = datetime.strptime(y, '%Y-%m-%d %H:%M:%S')

'2019-12-20 15:52:20'

In [8]:


#datetime_object.

In [9]:
#datetime_object
datetime_object.hour

15

In [10]:
#x.split()[0]

In [11]:
#x.split()[1].split('.')[0]

In [12]:
parsed_json= json.loads(df.loc[199][1])


In [13]:
parsed_json.keys()

dict_keys(['FaceGeometry', 'ThirdPartyException', 'FullBodyGeometry', 'UpperBodyGeometry', 'LowerBodyGeometry'])

In [14]:
t=parsed_json.get('ThirdPartyLibrariesFace')

In [15]:
parsed_json

{'FaceGeometry': '[{"X":945,"Y":252,"Width":47,"Height":47,"Top":252,"Bottom":298,"Left":945,"Right":991,"Location":{"X":945,"Y":252},"Size":{"Width":47,"Height":47},"TopLeft":{"X":945,"Y":252},"BottomRight":{"X":991,"Y":298}},{"X":857,"Y":471,"Width":72,"Height":72,"Top":471,"Bottom":542,"Left":857,"Right":928,"Location":{"X":857,"Y":471},"Size":{"Width":72,"Height":72},"TopLeft":{"X":857,"Y":471},"BottomRight":{"X":928,"Y":542}}]',
 'ThirdPartyException': 'System.Net.Http.HttpRequestException: Resource temporarily unavailable\n ---> System.Net.Sockets.SocketException (11): Resource temporarily unavailable\n   at System.Net.Http.ConnectHelper.ConnectAsync(String host, Int32 port, CancellationToken cancellationToken)\n   --- End of inner exception stack trace ---\n   at Microsoft.Rest.RetryDelegatingHandler.SendAsync(HttpRequestMessage request, CancellationToken cancellationToken)\n   at System.Net.Http.HttpClient.FinishSendAsyncBuffered(Task`1 sendTask, HttpRequestMessage request, Can

In [16]:
def as_complex(dct):
    if '__complex__' in dct:
        return complex(dct)
    return(dct)

In [17]:
attributes=['age','facialHair','gender','emotion','hair','glasses']

def get_attributes(json_data):
    faceId=json.loads(json.loads(json_data,object_hook=as_complex).get('MicrosoftData'),
           object_hook=as_complex)[0].get('faceId')
    faceAttributes=json.loads(json.loads(json_data,object_hook=as_complex).get('MicrosoftData'),
           object_hook=as_complex)[0].get('faceAttributes')
    age=faceAttributes.get(attributes[0])
    facialHair=max(faceAttributes.get(attributes[1]).items(),key=operator.itemgetter(1))[0]
    gender=faceAttributes.get(attributes[2])
    emotion=max(faceAttributes.get(attributes[3]).items(),key=operator.itemgetter(1))[0]
    #hair=max(faceAttributes.get(attributes[4]).items(),key=operator.itemgetter(1))[0]
    glasses=faceAttributes.get(attributes[5])
    return(pd.DataFrame({'faceId':faceId,
                       'age':age,'facialHair':facialHair,'glasses':glasses,
                       'gender':gender,'emotion':emotion},index=[0]))

In [20]:
error=[]
time=[]
final_df=pd.DataFrame()

for i in tqdm(range(df.shape[0])):
    j=i+1
#     if j % 10000==0:
#         print("Parsed :",j)
    parsed_json=json.loads(df.loc[i][1])
    if 'ThirdPartyLibrariesFace' in parsed_json.keys():
        text=parsed_json.get('ThirdPartyLibrariesFace')
        try:
            tmp_df=get_attributes(text)
            final_df=pd.concat([final_df,tmp_df])
            time.append(df[3][i])
        except:
            error.append(i)
    else:
        error.append(i)  

100%|██████████| 1000000/1000000 [38:08<00:00, 436.98it/s]  


In [22]:
final_df.shape


(74471, 6)

In [25]:
final_df

,faceId,age,facialHair,glasses,gender,emotion
0,d46a8115-c6a4-4b81-9b48-d91c276bd23a,40.0,moustache,noGlasses,female,neutral
0,017203ca-6f07-4fa6-99db-c0c883afe66c,46.0,moustache,noGlasses,male,neutral
0,2ea7604f-2ed8-4d5f-838c-464451aa4f93,49.0,moustache,readingGlasses,female,neutral
0,683566b0-5045-4bdf-bcdb-7c683a80536c,40.0,moustache,noGlasses,female,neutral
0,cf6994f1-4a66-4e8d-b32c-63b221aa62e5,62.0,moustache,readingGlasses,female,neutral
...,...,...,...,...,...,...
0,643a89e4-1a49-47a9-ba38-42aa28892969,23.0,moustache,noGlasses,female,neutral
0,b17e9811-4d80-46e2-8bfc-8a2c58df3352,31.0,moustache,noGlasses,female,happiness
0,a3916627-1f99-496b-88e0-17f0a5c8072a,20.0,beard,noGlasses,male,neutral
0,b58b0f59-38ec-47ef-80ba-0ecfa531eab7,20.0,moustache,noGlasses,male,neutral


In [33]:
final_df['time']=time

In [37]:
final_df['time'] = final_df['time'].apply(lambda x : x.split('.')[0])

In [42]:
final_df['datetime'] = final_df['time'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [53]:
# final_df['year'] = final_df['datetime'].apply(lambda x : x.year)
# final_df['month'] = final_df['datetime'].apply(lambda x : x.month )
final_df['day'] = final_df['datetime'].apply(lambda x : x.day)
final_df['hour'] = final_df['datetime'].apply(lambda x : x.hour)
#final_df['minute'] = final_df['datetime'].apply(lambda x : x.minute)
#final_df['second'] = final_df['datetime'].apply(lambda x : x.second)

In [64]:
final_df['emotion'].value_counts()

neutral      67143
happiness     5291
sadness       1002
surprise       560
anger          381
contempt        56
disgust         34
fear             4
Name: emotion, dtype: int64

In [66]:
final_df.shape

(74471, 14)

In [67]:
final_df.to_pickle('/Users/s0c02nj/Desktop/Face_attributes')

In [ ]:
# faceAttributes=json.loads(json.loads(t,object_hook=as_complex).get('MicrosoftData'),
#            object_hook=as_complex)[0].get('faceAttributes')

In [ ]:
# get_attributes(t)

In [ ]:
faceAttributes.get(attributes[4])

In [ ]:
max(faceAttributes.get(attributes[3]).items(),key=operator.itemgetter(1))[0]

In [ ]:
final_df.to_csv('/Users/s0p00zp/Desktop/Face_attributes.csv',index=False)

In [ ]:
final_df.head()